In [1]:
import os
import cv2
import numpy as np
import torch
import pandas as pd
from tqdm import tqdm
from segment_anything import sam_model_registry, SamPredictor
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import math
import re
import traceback
from sklearn.model_selection import train_test_split
import albumentations as A
import glob
import gc

In [2]:

cuda_available = torch.cuda.is_available()
print(f"CUDA доступен: {cuda_available}")

# Если CUDA доступен, можно проверить количество устройств и версию
if cuda_available:
    device_count = torch.cuda.device_count()
    print(f"Количество доступных CUDA устройств: {device_count}")

    # Опционально: проверить имя устройства
    if device_count > 0:
        print(f"Имя текущего CUDA устройства: {torch.cuda.get_device_name(0)}")

    # Опционально: проверить версию CUDA, с которой был собран PyTorch
    print(f"Версия CUDA, с которой собран PyTorch: {torch.version.cuda}")

CUDA доступен: True
Количество доступных CUDA устройств: 1
Имя текущего CUDA устройства: NVIDIA GeForce RTX 3050 Ti Laptop GPU
Версия CUDA, с которой собран PyTorch: 12.8


In [3]:
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='NVIDIA GeForce RTX 3050 Ti Laptop GPU', major=8, minor=6, total_memory=4095MB, multi_processor_count=20, uuid=aaf385ef-3536-493b-3bb9-b3803478ad7c, pci_bus_id=1, pci_device_id=0, pci_domain_id=0, L2_cache_size=2MB)

In [4]:

SAM_CHECKPOINT = "C:/Users/user/Desktop/data/sam/checkpoint/sam_vit_h_4b8939.pth" 
MODEL_TYPE = "vit_h"  
DEVICE = torch.device("cuda")
BATCH_SIZE = 1
LEARNING_RATE = 1e-4
NUM_EPOCHS = 10
IMAGE_SIZE = (1024,1024)

DRAWINGS_DIR = "C:/Users/user/Desktop/Work/DeepLearning/JCB/images_dataset/label"
MASKS_DIR = "C:/Users/user/Desktop/valid"
OUTPUT_DIR = "C:/Users/user/Desktop/data/sam"
CSV_PATH = "C:/Users/user/Desktop/data/merged_result.csv"

In [5]:
def parse_shape_path(shape_path):
    tokens = re.findall(r'[mlxe]|\d+,\d+', shape_path, flags=re.IGNORECASE)
    min_x, max_x = math.inf, -math.inf
    min_y, max_y = math.inf, -math.inf
    found_coords = False 
    i = 0
    while i < len(tokens):
        token = tokens[i].lower()
        if token == 'm' or token == 'l':
            i += 1
            if i < len(tokens) and ',' in tokens[i]:
                try:
                    x, y = map(float, tokens[i].split(','))
                    min_x = min(min_x, x)
                    max_x = max(max_x, x)
                    min_y = min(min_y, y)
                    max_y = max(max_y, y)
                    found_coords = True
                except ValueError:
                    pass
        i += 1
    if not found_coords:
        print("Предупреждение: В строке пути не найдено валидных координат.")
        return None
    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    return (center_x, center_y)
def check_cuda():
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA device count: {torch.cuda.device_count()}")
        print(f"CUDA device name: {torch.cuda.get_device_name(0)}")


In [6]:
def optimize_cuda_memory():
    import gc
    import torch
    
    # Force garbage collection
    gc.collect()
    torch.cuda.empty_cache()
    
    # Lower memory usage settings
    torch.cuda.set_per_process_memory_fraction(0.8, 0)  # Lower to 80%
    
    # Set PyTorch to use more efficient memory allocator
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
    
    # Use deterministic algorithms if possible (can reduce memory usage)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    print("Applied aggressive memory optimizations")

In [7]:

class DrawingMaskDataset(Dataset):
    """Набор данных для обучения модели сегментации."""
    def __init__(self, csv_path, drawings_dir, masks_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.drawings_dir = drawings_dir
        self.masks_dir = masks_dir
        self.transform = transform
        drawing_extensions = ['.tiff']
        mask_extensions = ['.tiff']
        valid_entries = []
        print("Validating dataset entries...")
        file_cache = {}
        for _, row in tqdm(self.data.iterrows(), total=len(self.data)):
            drawing_id = row['drawing_name']
            drawing_path = None
            for ext in drawing_extensions:
                path = os.path.join(self.drawings_dir, f"{drawing_id}{ext}")
                if path not in file_cache:
                    file_cache[path] = os.path.exists(path)
                if file_cache[path]:
                    drawing_path = path
                    break
            mask_path = None
            for ext in mask_extensions:
                path = os.path.join(self.masks_dir, f"{drawing_id}{ext}")
                if path not in file_cache:
                    file_cache[path] = os.path.exists(path)
                if file_cache[path]:
                    mask_path = path
                    break
            if drawing_path and mask_path:
                valid_entries.append({
                    'drawing_name': drawing_id,
                    'drawing_path': drawing_path,
                    'mask_path': mask_path,
                    'ShapePath': row['ShapePath']
                })
        self.valid_data = pd.DataFrame(valid_entries)
        print(f"Found {len(self.valid_data)} valid entries out of {len(self.data)}")
        self.data = None
        
    def __len__(self):
        return len(self.valid_data)
    
    def __getitem__(self, idx):
        try:
            row = self.valid_data.iloc[idx]
            drawing_path = row['drawing_path']
            mask_path = row['mask_path']
            image = cv2.imread(drawing_path)
            if image is None:
                print(f"Failed to read image: {drawing_path}")
                image = np.zeros((IMAGE_SIZE[0], IMAGE_SIZE[1], 3), dtype=np.uint8)
            else:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            if mask is None:
                print(f"Failed to read mask: {mask_path}")
                mask = np.zeros((IMAGE_SIZE[0], IMAGE_SIZE[1]), dtype=np.uint8)

            if image.shape[:2] != mask.shape[:2]:
                print(f"Resizing mask to match image for {row['drawing_name']}")
                mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)
                
            mask = (mask > 0).astype(np.float32)
            shape_path = str(row['ShapePath']).strip()
            x_center, y_center = parse_shape_path(shape_path)
            point_coords = np.array([[x_center, y_center]])
            point_labels = np.array([1])
            if self.transform:
                transformed = self.transform(image=image, mask=mask, keypoints=point_coords)
                image = transformed['image']
                mask = transformed['mask']
                point_coords = np.array(transformed['keypoints'])
            image_tensor = torch.from_numpy(image.transpose(2, 0, 1)).float() / 255.0
            mask_tensor = torch.from_numpy(mask).float()
            point_coords_tensor = torch.from_numpy(point_coords).float()
            point_labels_tensor = torch.from_numpy(point_labels).float()
            return {
                'image': image_tensor,
                'mask': mask_tensor,
                'point_coords': point_coords_tensor,
                'point_labels': point_labels_tensor,
                'drawing_id': row['drawing_name']
            }
        except Exception as e:
            print(f"Error in __getitem__[{idx}]: {e}")
            traceback.print_exc()
            return {
                'image': torch.zeros((3, IMAGE_SIZE[0], IMAGE_SIZE[1]), dtype=torch.float32),
                'mask': torch.zeros((IMAGE_SIZE[0], IMAGE_SIZE[1]), dtype=torch.float32),
                'point_coords': torch.tensor([[IMAGE_SIZE[0]/2, IMAGE_SIZE[1]/2]], dtype=torch.float32),
                'point_labels': torch.tensor([1.0], dtype=torch.float32),
                'drawing_id': f"error_{idx}"
            }

class EmbeddingsDataset(Dataset):
    def __init__(self, original_dataset, embeddings_dir):
        self.original_dataset = original_dataset
        self.embeddings_dir = embeddings_dir
        self.embedding_files = {}
        if os.path.exists(embeddings_dir):
            print("Indexing embeddings...")
            embedding_pattern = os.path.join(embeddings_dir, "*_embedding.pt")
            for embedding_path in tqdm(glob.glob(embedding_pattern)):
                filename = os.path.basename(embedding_path)
                drawing_id = filename.replace("_embedding.pt", "")
                self.embedding_files[drawing_id] = embedding_path         
        print(f"Found {len(self.embedding_files)} precomputed embeddings")
    def __len__(self):
        return len(self.original_dataset)
    def __getitem__(self, idx):
        item = self.original_dataset[idx]
        drawing_id = item['drawing_id']
        item['use_precomputed_embedding'] = False
        if drawing_id in self.embedding_files:
            try:
                embedding_path = self.embedding_files[drawing_id]
                item['image_embedding'] = torch.load(embedding_path, map_location='cpu')
                item['use_precomputed_embedding'] = True
            except Exception as e:
                print(f"Error loading embedding for {drawing_id}: {e}")
        return item


In [8]:
from contextlib import nullcontext

def precompute_embeddings(dataloader, sam_model, device, output_dir):
    """Предварительно вычисляет эмбеддинги для всех изображений и сохраняет их на диск."""
    print(f"Precomputing embeddings for {len(dataloader)} batches...")
    
    # Make sure the entire model is on the specified device
    sam_model = sam_model.to(device)
    sam_model.eval()
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(dataloader, desc="Computing embeddings")):
            try:
                images = batch['image'].to(device)
                drawing_ids = batch['drawing_id']
                
                # Get model's expected input dimensions
                input_size = 1024
                
                # Process each image in the batch
                for i, (image, drawing_id) in enumerate(zip(images, drawing_ids)):
                    # Skip if embedding already exists
                    embedding_path = os.path.join(output_dir, f"{drawing_id}_embedding.pt")
                    if os.path.exists(embedding_path):
                        continue
                    
                    # Process the image while keeping it on the device
                    img = image.cpu().numpy().transpose(1, 2, 0)  # Convert to HWC format
                    
                    # Convert back to tensor - ensure it's on the same device as the model
                    tensor_img = torch.from_numpy(img.transpose(2, 0, 1)).float().unsqueeze(0).to(device)
                    
                    # Compute embedding
                    try:
                        # Use appropriate context based on device
                        context_mgr = torch.cuda.amp.autocast() if device == "cuda" else nullcontext()
                        with context_mgr:
                            embedding = sam_model.image_encoder(tensor_img)
                        
                        # Save embedding
                        torch.save(embedding.squeeze(0).cpu(), embedding_path)
                    except RuntimeError as e:
                        if "out of memory" in str(e):
                            print(f"CUDA OOM for {drawing_id}. Trying on CPU...")
                            # Move both model and tensor to CPU
                            temp_encoder = sam_model.image_encoder.to("cpu")
                            cpu_tensor_img = tensor_img.to("cpu")
                            
                            # Compute on CPU
                            embedding = temp_encoder(cpu_tensor_img)
                            torch.save(embedding.squeeze(0).cpu(), embedding_path)
                            
                            # Move encoder back to original device
                            sam_model.image_encoder.to(device)
                            torch.cuda.empty_cache()
                        else:
                            print(f"Error computing embedding for {drawing_id}: {e}")
                            continue
                
                # Clear memory
                if device == "cuda":
                    torch.cuda.empty_cache()
                
                # Log progress
                if batch_idx % 10 == 0:
                    print(f"Processed {batch_idx}/{len(dataloader)} batches")
            
            except Exception as e:
                print(f"Error in batch {batch_idx}: {e}")
                traceback.print_exc()
                continue

In [ ]:
import torch.nn.functional as fa
from torch.utils.checkpoint import checkpoint
                
class SAMFineTuner:
    """Класс для файнтюнинга SAM на размеченных данных с градиентным чекпойнтингом и аккумуляцией."""
    def __init__(self, checkpoint_path, model_type, 
             gradient_accumulation_steps=4,
             use_gradient_checkpointing=True):
        print("Initializing SAM fine tuner with memory optimizations...")
        self.device = DEVICE
        
        # Load model in half precision to reduce memory usage
        with torch.cuda.amp.autocast():
            self.sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
            
            # First move to CPU
            self.sam.to("cpu")
            
            # Apply gradient checkpointing before moving to GPU
            if use_gradient_checkpointing:
                print("Enabling gradient checkpointing...")
  
                # Store the original forward method
                original_forward = self.sam.mask_decoder.forward
                
                # Define properly parameterized forward method with checkpointing
                def forward_with_checkpoint(image_embeddings, image_pe, sparse_prompt_embeddings, 
                                        dense_prompt_embeddings, multimask_output):
                    def custom_forward(image_embeddings, image_pe, sparse_prompt_embeddings, 
                                    dense_prompt_embeddings, multimask_output):
                        return original_forward(
                            image_embeddings, 
                            image_pe, 
                            sparse_prompt_embeddings, 
                            dense_prompt_embeddings, 
                            multimask_output
                        )
                    
                    return checkpoint(
                        custom_forward, 
                        image_embeddings, 
                        image_pe, 
                        sparse_prompt_embeddings, 
                        dense_prompt_embeddings, 
                        multimask_output,
                        use_reentrant=False
                    )
                
                # Replace the forward method with our checkpointed version
                self.sam.mask_decoder.forward = forward_with_checkpoint
            
            # Freeze encoder before moving to GPU to save memory
            for param in self.sam.image_encoder.parameters():
                param.requires_grad = False
            
            # Now move to GPU in parts
            try:
                # Try loading components separately to manage memory better
                self.sam.mask_decoder.to(self.device)
                self.sam.prompt_encoder.to(self.device)
                # Keep image_encoder on CPU if possible
                # We'll move embeddings as needed during processing
            except torch.cuda.OutOfMemoryError:
                print("Failed to load model to GPU. Attempting CPU fallback...")
                self.device = "cpu"  # Fall back to CPU
                
        self.optimizer = optim.Adam(
            self.sam.mask_decoder.parameters(),
            lr=LEARNING_RATE
        )
        
        # Use mixed precision training
        self.scaler = torch.amp.GradScaler()
        
        # Gradient accumulation parameters
        self.gradient_accumulation_steps = gradient_accumulation_steps
        print(f"Using gradient accumulation with {gradient_accumulation_steps} steps")
        
        self.dice_loss = self.dice_loss_fn
        self.bce_loss = nn.BCEWithLogitsLoss()
        
        self.best_model_path = os.path.join(OUTPUT_DIR, "best_sam_model.pth")
        print("SAM fine tuner initialized with memory optimizations")

    def focal_loss(self, pred, target, alpha=0.8, gamma=2.0):
        """Focal Loss для лучшей работы с несбалансированными масками."""
        bce_loss = fa.binary_cross_entropy_with_logits(pred, target, reduction='none')
        pt = torch.exp(-bce_loss)
        focal_weight = alpha * (1 - pt) ** gamma
        loss = focal_weight * bce_loss
        return loss.mean()

    def dice_loss_fn(self, pred, target):
        """Улучшенная функция потери Dice с проверками."""
        smooth = 1.0
        pred = torch.sigmoid(pred)
        
        # Добавляем отладочную информацию
        if torch.isnan(pred).any() or torch.isinf(pred).any():
            print("Warning: NaN or Inf values in predictions after sigmoid!")
        
        # Можно добавить проверку на пустые маски
        empty_masks = target.sum(dim=(1, 2)) == 0
        
        intersection = (pred * target).sum(dim=(1, 2))
        union = pred.sum(dim=(1, 2)) + target.sum(dim=(1, 2))
        
        # Вычисление Dice с обработкой краевых случаев
        dice = (2. * intersection + smooth) / (union + smooth)
        
        # Если все маски пустые, избегаем нулевого значения
        if empty_masks.all():
            print("Warning: All target masks are empty!")
            # Можно вернуть фиксированное значение или другую метрику
        
        loss = 1 - dice.mean()
        
        # Дополнительные проверки
        if loss.item() == 0:
            print("Dice loss is exactly 0!")
            print(f"Intersection: {intersection.mean().item()}, Union: {union.mean().item()}")

        return loss
    
    def combined_loss(self, pred, target):
        """Комбинированная функция потерь."""
        dice = self.dice_loss_fn(pred, target)
        bce = fa.binary_cross_entropy_with_logits(pred, target)
        focal = self.focal_loss(pred, target)
        return 0.25 * dice + 0.25 * bce + 0.5 * focal
    

    def train_epoch(self, dataloader):
        """Обучение модели на одной эпохе с аккумуляцией градиентов и смешанной точностью."""
        print("Starting train epoch with gradient accumulation and mixed precision...")
        self.sam.train()
        total_loss = 0
        processed_batches = 0
        
        # Обнуляем градиенты перед началом эпохи
        self.optimizer.zero_grad()
        
        progress_bar = tqdm(dataloader, desc="Training")
        for batch_idx, batch in enumerate(progress_bar):
            try:
                # Вычисляем нормализующий множитель для шага аккумуляции
                normalize_factor = 1.0 / self.gradient_accumulation_steps
                
                # Определяем, используем ли предварительно вычисленные эмбеддинги
                if 'use_precomputed_embedding' in batch and batch['use_precomputed_embedding'][0]:
                    image_embeddings = batch['image_embedding'].to(self.device)
                    masks = batch['mask'].to(self.device)
                    point_coords = batch['point_coords'].to(self.device)
                    point_labels = batch['point_labels'].to(self.device)
                    point_labels = point_labels.to(torch.int64)
                else:
                    images = batch['image'].to(self.device)
                    masks = batch['mask'].to(self.device)
                    point_coords = batch['point_coords'].to(self.device)
                    point_labels = batch['point_labels'].to(self.device)
                    point_labels = point_labels.to(torch.int64)
                    
                    with torch.no_grad():
                        # Move image encoder to device temporarily if needed
                        if next(self.sam.image_encoder.parameters()).device != self.device:
                            self.sam.image_encoder.to(self.device)
                            
                        with torch.cuda.amp.autocast():
                            image_embeddings = self.sam.image_encoder(images)
                        
                        # Move back to CPU if we're memory constrained
                        if self.device == "cuda" and torch.cuda.memory_allocated() > 0.9 * torch.cuda.get_device_properties(0).total_memory:
                            self.sam.image_encoder.to("cpu")
                            torch.cuda.empty_cache()
                
                # Use mixed precision for forward and backward passes
                with torch.cuda.amp.autocast():
                    # Получаем эмбеддинги для промптов
                    sparse_embeddings, dense_embeddings = self.sam.prompt_encoder(
                        points=(point_coords, point_labels),
                        boxes=None,
                        masks=None
                    )
                    
                    # Используем декодер для предсказания масок
                    mask_predictions, _ = self.sam.mask_decoder(
                        image_embeddings=image_embeddings,
                        image_pe=self.sam.prompt_encoder.get_dense_pe(),
                        sparse_prompt_embeddings=sparse_embeddings,
                        dense_prompt_embeddings=dense_embeddings,
                        multimask_output=False,
                    )
                    
                    # Изменяем размер предсказаний до размера истинных масок
                    target_h, target_w = masks.shape[-2:]
                    upscaled_predictions = fa.interpolate(
                        mask_predictions,
                        size=(target_h, target_w),
                        mode='bilinear',
                        align_corners=False
                    )
                    
                    # Вычисляем потери
                    combined_loss = self.combined_loss(upscaled_predictions, masks.unsqueeze(1))

                    if batch_idx % 100 == 0: 

                        print(f"Combined loss: {combined_loss.detach().item()}")
                        
                    loss = (combined_loss) * normalize_factor
                                                        
                # Обратное распространение ошибки с смешанной точностью
                self.scaler.scale(loss).backward()
                
                # Обработка проблем с бесконечными значениями
                if not torch.isfinite(loss).all():
                    print(f"Batch {batch_idx}: Warning! Non-finite loss value: {loss.item()}")
                
                # Обновляем веса, только если достигли нужного количества шагов аккумуляции
                # или если это последний батч
                is_last_batch = batch_idx == len(dataloader) - 1
                if (batch_idx + 1) % self.gradient_accumulation_steps == 0 or is_last_batch:
                    # Градиентное клиппирование для предотвращения взрыва градиентов
                    self.scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(self.sam.mask_decoder.parameters(), max_norm=1.0)
                    
                    # Update weights with scaled gradients
                    self.scaler.step(self.optimizer)
                    self.scaler.update()
                    self.optimizer.zero_grad()
                    
                    # Free up memory
                    torch.cuda.empty_cache()
                
                # Учитываем потери
                actual_loss = loss.item() * self.gradient_accumulation_steps
                total_loss += actual_loss
                processed_batches += 1
                
                if batch_idx % 10 == 0:
                    progress_bar.set_postfix({"Loss": f"{actual_loss:.4f}"})
                    
            except Exception as e:
                print(f"Error in batch {batch_idx}: {e}")
                traceback.print_exc()
                continue
                
        return total_loss / max(processed_batches, 1)
    
    def validate(self, dataloader):
        """Валидация модели."""
        print("Starting validation...")
        self.sam.eval()
        total_loss = 0
        processed_batches = 0
        progress_bar = tqdm(dataloader, desc="Validation")
        
        with torch.no_grad(): 
            for batch_idx, batch in enumerate(progress_bar):
                try:
                    masks = batch['mask'].to(self.device)
                    point_coords = batch['point_coords'].to(self.device)
                    point_labels = batch['point_labels'].to(self.device)
                    point_labels = point_labels.to(torch.int64)
                    
                    if 'use_precomputed_embedding' in batch and batch['use_precomputed_embedding'][0]:
                        image_embeddings = batch['image_embedding'].to(self.device)
                    else:
                        images = batch['image'].to(self.device)
                        image_embeddings = self.sam.image_encoder(images)
                        
                    sparse_embeddings, dense_embeddings = self.sam.prompt_encoder(
                        points=(point_coords, point_labels),
                        boxes=None,
                        masks=None
                    )
                    
                    mask_predictions, _ = self.sam.mask_decoder(
                        image_embeddings=image_embeddings,
                        image_pe=self.sam.prompt_encoder.get_dense_pe(),
                        sparse_prompt_embeddings=sparse_embeddings,
                        dense_prompt_embeddings=dense_embeddings,
                        multimask_output=False,
                    )
                    
                    target_h, target_w = masks.shape[-2:]
                    upscaled_predictions = fa.interpolate(
                        mask_predictions,
                        size=(target_h, target_w),
                        mode='bilinear',
                        align_corners=False
                    )
                    
                    # pred_masks = torch.sigmoid(upscaled_predictions) > 0.5

                    # intersection = (pred_masks.squeeze(1) * masks).sum((1, 2))
                    # union = pred_masks.squeeze(1).sum((1, 2)) + masks.sum((1, 2)) - intersection
                    # dice = (2. * intersection) / (union + 1e-6)

                    batch_loss = self.combined_loss(upscaled_predictions, masks.unsqueeze(1)) # Вызываем combined_loss

                    total_loss += batch_loss.item() 
  
                    # batch_dice = combined.mean().item()
                    # total_dice += batch_dice
                    processed_batches += 1
                    
                    if batch_idx % 10 == 0:
                        progress_bar.set_postfix({"Dice": f"{batch_loss:.4f}"})
                        
                        
                except Exception as e:
                    print(f"Error in validation batch {batch_idx}: {e}")
                    traceback.print_exc()
                    continue
                    
        return total_loss / max(processed_batches, 1)
    
    def train(self, train_loader, val_loader, num_epochs):
        """Обучение модели на указанное количество эпох."""
        print(f"Starting training for {num_epochs} epochs with memory optimizations...")
        best_val = 10
        
        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch+1}/{num_epochs}")
            
            # Обучение на текущей эпохе
            train_loss = self.train_epoch(train_loader)
            
            # Валидация после эпохи обучения
            val = self.validate(val_loader)
            
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Dice: {val:.4f}")
            
            # Сохраняем модель, только если она лучше предыдущих
            if val < best_val:
                best_val = val
                torch.save(
                    self.sam.state_dict(), 
                    self.best_model_path
                )
                print(f"Saved best model with Val Dice: {val:.4f}")
    
    def load_best_model(self):
        """Загрузка лучшей модели."""
        best_model_path = os.path.join(OUTPUT_DIR, "best_sam_model.pth")

        # *** ДОБАВЛЕНО: Убедиться, что путь к файлу весов сохранен как self.best_model_path в __init__ или другом месте ***
        # Или просто используйте локальную переменную best_model_path, как в вашем примере кода

        if os.path.exists(best_model_path):
            # --- Шаг 1: Очистка кэша CUDA ---
            print("Clearing CUDA cache before loading model...")
            torch.cuda.empty_cache() # Освобождаем неиспользуемую зарезервированную память

            # --- Шаг 2: Попытка загрузить state_dict на CPU сначала ---
            # Если даже это не сработает, возможно, у вас проблемы с RAM или файл поврежден.
            try:
                print(f"Attempting to load model state dict from {best_model_path} onto CPU...")
                state_dict_cpu = torch.load(best_model_path, map_location='cpu')
                print("State dict loaded onto CPU.")

                # --- Шаг 3: Загрузка state_dict в модель на GPU ---
                # Модель (self.sam) уже должна быть на self.device (GPU)
                print(f"Loading state dict into model on {self.device}...")
                self.sam.load_state_dict(state_dict_cpu)
                print(f"Loaded best model from {best_model_path}")

                # --- Опционально: Очистка CPU state_dict после загрузки ---
                del state_dict_cpu
                # После загрузки на GPU, можно еще раз очистить кэш на всякий случай
                torch.cuda.empty_cache()

            except Exception as e:
                print(f"Error loading model state dict: {e}")
                import traceback # Убедитесь, что импортирован
                traceback.print_exc()
                print("Could not load best model. Using current model (if initialized).")

        else:
            print(f"Best model not found at {best_model_path}. Using current model (potentially from checkpoint if provided in init).")
    
    def predict_mask(self, image, point_coords, embeddings_dir=None, drawing_id=None):
        """Предсказывает маску для изображения с указанной точкой с оптимизированным управлением памятью."""
        self.sam.eval()
        
        # Определяем целевое устройство заранее на основе доступной памяти
        if self.device == "cuda":
            available_memory = torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated()
            target_device = "cuda" if available_memory > 3 * 1024 * 1024 * 1024 else "cpu"  # Используем GPU если доступно >3GB
            torch.cuda.empty_cache()  # Очищаем кэш перед началом
        else:
            target_device = "cpu"
        
        print(f"Selected {target_device} as target device for prediction")
        
        # Конвертируем изображение в нужный формат
        if image.shape[2] == 3:
            if image.dtype == np.uint8:
                rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            else:
                rgb_image = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_BGR2RGB)
        else:
            rgb_image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        
        original_h, original_w = image.shape[:2]
        
        # Уменьшаем изображение для энкодера, если оно слишком большое
        max_size = 1024
        if original_h > max_size or original_w > max_size:
            scale_factor = max_size / max(original_h, original_w)
            new_h, new_w = int(original_h * scale_factor), int(original_w * scale_factor)
            rgb_image_resized = cv2.resize(rgb_image, (new_w, new_h), interpolation=cv2.INTER_AREA)
        else:
            rgb_image_resized = rgb_image
        
        # Пытаемся использовать предварительно вычисленные эмбеддинги
        image_embeddings = None
        use_precomputed = False
        
        if embeddings_dir and drawing_id:
            embedding_path = os.path.join(embeddings_dir, f"{drawing_id}_embedding.pt")
            if os.path.exists(embedding_path):
                try:
                    image_embeddings = torch.load(embedding_path, map_location="cpu")
                    image_embeddings = image_embeddings.unsqueeze(0)
                    use_precomputed = True
                    print("Using precomputed embeddings")
                except Exception as e:
                    print(f"Error loading embedding: {e}")
                    use_precomputed = False
        
        # Если нет предварительных эмбеддингов, вычисляем их
        if not use_precomputed:
            # Подготавливаем тензор изображения на CPU
            tensor_image = torch.from_numpy(rgb_image_resized.transpose(2, 0, 1)).float() / 255.0
            tensor_image = tensor_image.unsqueeze(0)
            
            # Определяем устройство для энкодера
            encoder_device = target_device
            
            try:
                # Размещаем энкодер и изображение на выбранном устройстве
                self.sam.image_encoder.to(encoder_device)
                tensor_image = tensor_image.to(encoder_device)
                
                # Вычисляем эмбеддинги с помощью автоматического смешанного типа для экономии памяти
                with torch.no_grad():
                    if encoder_device == "cuda":
                        with torch.cuda.amp.autocast():
                            image_embeddings = self.sam.image_encoder(tensor_image)
                    else:
                        image_embeddings = self.sam.image_encoder(tensor_image)
                        
                # Сохраняем вычисленные эмбеддинги если требуется
                if embeddings_dir and drawing_id:
                    os.makedirs(embeddings_dir, exist_ok=True)
                    embedding_path = os.path.join(embeddings_dir, f"{drawing_id}_embedding.pt")
                    # Сохраняем на CPU для универсальности
                    torch.save(image_embeddings.squeeze(0).cpu(), embedding_path)
                    
            except RuntimeError as e:
                # Обработка ошибки памяти
                if "out of memory" in str(e):
                    print(f"Memory error during encoding. Falling back to CPU with smaller image...")
                    
                    # Очищаем память
                    if encoder_device == "cuda":
                        self.sam.image_encoder.to("cpu")
                        torch.cuda.empty_cache()
                    
                    # Уменьшаем изображение еще сильнее
                    emergency_scale = 0.5
                    smaller_h, smaller_w = int(rgb_image_resized.shape[0] * emergency_scale), int(rgb_image_resized.shape[1] * emergency_scale)
                    rgb_image_small = cv2.resize(rgb_image_resized, (smaller_w, smaller_h), interpolation=cv2.INTER_AREA)
                    
                    # Создаем новый тензор на CPU
                    tensor_image = torch.from_numpy(rgb_image_small.transpose(2, 0, 1)).float() / 255.0
                    tensor_image = tensor_image.unsqueeze(0)
                    
                    # Вычисляем эмбеддинги на CPU
                    with torch.no_grad():
                        image_embeddings = self.sam.image_encoder(tensor_image)
                else:
                    raise e
            finally:
                # Очищаем память в любом случае
                del tensor_image
                if encoder_device == "cuda":
                    torch.cuda.empty_cache()
        
        # === Подготовка к предсказанию маски ===
        # Переводим все необходимые компоненты на целевое устройство за один раз
        # для уменьшения фрагментации памяти
        
        try:
            # Подготавливаем точки и метки
            tensor_points = torch.tensor(point_coords, dtype=torch.float).unsqueeze(0)
            tensor_labels = torch.ones(1, point_coords.shape[0], dtype=torch.int)
            
            # Перемещаем все компоненты на целевое устройство
            components_moved = False
            
            try:
                # Все перемещаем на целевое устройство за один раз
                self.sam.prompt_encoder.to(target_device)
                self.sam.mask_decoder.to(target_device)
                tensor_points = tensor_points.to(target_device)
                tensor_labels = tensor_labels.to(target_device)
                image_embeddings = image_embeddings.to(target_device)
                components_moved = True
                
                # Предсказание маски полностью на целевом устройстве
                with torch.no_grad():
                    # Вычисляем эмбеддинги промптов
                    sparse_embeddings, dense_embeddings = self.sam.prompt_encoder(
                        points=(tensor_points, tensor_labels),
                        boxes=None,
                        masks=None
                    )
                    
                    # Получаем image_pe
                    image_pe = self.sam.prompt_encoder.get_dense_pe()
                    
                    # Освобождаем промежуточную память
                    del tensor_points, tensor_labels
                    if target_device == "cuda":
                        torch.cuda.empty_cache()
                    
                    # Генерируем предсказания маски
                    mask_predictions, _ = self.sam.mask_decoder(
                        image_embeddings=image_embeddings,
                        image_pe=image_pe,
                        sparse_prompt_embeddings=sparse_embeddings,
                        dense_prompt_embeddings=dense_embeddings,
                        multimask_output=False,
                    )
                    
                    # Освобождаем промежуточную память
                    del image_embeddings, sparse_embeddings, dense_embeddings, image_pe
                    if target_device == "cuda":
                        torch.cuda.empty_cache()
                    
                    # Возвращаем предсказания на CPU для интерполяции
                    mask_predictions = mask_predictions.cpu()
                    
                    # Интерполяция до оригинального размера
                    upscaled_predictions = fa.interpolate(
                        mask_predictions,
                        size=(original_h, original_w),
                        mode="bilinear",
                        align_corners=False
                    )
                    
                    # Преобразуем в маску
                    pred_mask = torch.sigmoid(upscaled_predictions) > 0.5
                    pred_mask = pred_mask.squeeze().numpy().astype(np.uint8) * 255
                    
                    # Освобождаем финальную память
                    del mask_predictions, upscaled_predictions
            
            except RuntimeError as e:
                if "out of memory" in str(e) and components_moved:
                    # Если произошла ошибка памяти после перемещения компонентов,
                    # переход на CPU уже обрабатывается в следующем блоке catch
                    raise e
                elif "out of memory" in str(e):
                    # Если произошла ошибка во время перемещения, перемещаем все на CPU
                    print("Out of memory during component movement. Using CPU for all processing.")
                    target_device = "cpu"
                    raise RuntimeError("Retry on CPU")
                else:
                    raise e

        except (RuntimeError, Exception) as e:
            # Если возникает ошибка или мы явно запрашиваем повторную попытку на CPU
            if isinstance(e, RuntimeError) and "Retry on CPU" in str(e) or "out of memory" in str(e):
                print("Falling back to complete CPU processing for prediction")
                
                # Перемещаем все на CPU и очищаем GPU память
                self.sam.prompt_encoder.to("cpu")
                self.sam.mask_decoder.to("cpu")
                if hasattr(torch.cuda, 'empty_cache'):
                    torch.cuda.empty_cache()
                
                # Убеждаемся, что эмбеддинги на CPU
                image_embeddings = image_embeddings.cpu() if image_embeddings is not None else None
                
                # Подготавливаем точки и метки на CPU
                tensor_points = torch.tensor(point_coords, dtype=torch.float).unsqueeze(0)
                tensor_labels = torch.ones(1, point_coords.shape[0], dtype=torch.int)
                
                with torch.no_grad():
                    # Повторяем все шаги на CPU
                    sparse_embeddings, dense_embeddings = self.sam.prompt_encoder(
                        points=(tensor_points, tensor_labels),
                        boxes=None,
                        masks=None
                    )
                    
                    image_pe = self.sam.prompt_encoder.get_dense_pe()
                    
                    mask_predictions, _ = self.sam.mask_decoder(
                        image_embeddings=image_embeddings,
                        image_pe=image_pe,
                        sparse_prompt_embeddings=sparse_embeddings,
                        dense_prompt_embeddings=dense_embeddings,
                        multimask_output=False,
                    )
                    
                    # Выполняем интерполяцию на CPU
                    upscaled_predictions = fa.interpolate(
                        mask_predictions,
                        size=(original_h, original_w),
                        mode="bilinear",
                        align_corners=False
                    )
                    
                    pred_mask = torch.sigmoid(upscaled_predictions) > 0.5
                    pred_mask = pred_mask.squeeze().numpy().astype(np.uint8) * 255
                    
                    # Очищаем использованную память
                    del tensor_points, tensor_labels, sparse_embeddings, dense_embeddings
                    del image_pe, mask_predictions, upscaled_predictions
            else:
                # Если это не ошибка памяти, пробрасываем ее дальше
                raise e
        
        finally:
            # Всегда перемещаем все компоненты обратно на CPU для уменьшения использования GPU памяти
            self.sam.prompt_encoder.to("cpu") 
            self.sam.mask_decoder.to("cpu")
            self.sam.image_encoder.to("cpu")
            if hasattr(torch.cuda, 'empty_cache'):
                torch.cuda.empty_cache()
        
        return pred_mask
     

In [ ]:
def generate_all_masks(df, fine_tuner, embeddings_dir=None):
    """Генерирует маски для всех чертежей из DataFrame."""
    print("Generating masks for all drawings...")
    masks_output_dir = os.path.join(OUTPUT_DIR, "generated_masks")
    os.makedirs(masks_output_dir, exist_ok=True)
 
    transform_resize = A.Compose([
        A.Resize(IMAGE_SIZE[0], IMAGE_SIZE[1]),
    ], keypoint_params=A.KeypointParams(format='xy'))  

    drawing_paths_cache = {}
    drawing_groups = df.groupby('drawing_name')

    for drawing_id, group in tqdm(drawing_groups, desc="Processing drawings"):
        try:
 
            if drawing_id in drawing_paths_cache:
                drawing_path = drawing_paths_cache[drawing_id]
            else:
                drawing_path = None
                for ext in ['.tiff', '.TIF', '.tif', '.png']:
                    path = os.path.join(DRAWINGS_DIR, f"{drawing_id}{ext}")
                    if os.path.exists(path):
                        drawing_path = path
                        drawing_paths_cache[drawing_id] = path
                        break

            if drawing_path is None:
                print(f"Warning: Drawing for {drawing_id} not found. Skipping.")
                continue

            original_image = cv2.imread(drawing_path)  
            if original_image is None:
                print(f"Failed to read image: {drawing_path}")
                continue
 
            resized_image = transform_resize(image=original_image)['image']  
 
            original_points = []
            for _, row in group.iterrows():
                 try:
                     shape_path = row['ShapePath']
                     x, y = parse_shape_path(shape_path)
                     original_points.append((x, y)) 
                 except Exception as e:
                     print(f"Error processing point: {e}")
                     continue

            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

            original_h, original_w = original_image.shape[:2]
            target_h, target_w = IMAGE_SIZE  
 
            scale_x = target_w / original_w
            scale_y = target_h / original_h

            for i, (orig_x, orig_y) in enumerate(original_points): 
                try:
                    mask_filename = f"{drawing_id}.tiff"
                    mask_path = os.path.join(masks_output_dir, mask_filename)

                    if os.path.exists(mask_path):
                        print(f"Mask for {drawing_id} part {i+1} already exists. Skipping.")
                        continue

                    # *** МАСШТАБИРУЕМ КООРДИНАТЫ ТОЧКИ ***
                    scaled_x = orig_x * scale_x
                    scaled_y = orig_y * scale_y
                    scaled_point_coords = np.array([[scaled_x, scaled_y]], dtype=np.float32) # Формат для predict_mask
                
                    mask = fine_tuner.predict_mask(resized_image, scaled_point_coords, embeddings_dir, drawing_id)
               
                    mask_clean = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

                  
                    cv2.imwrite(mask_path, mask_clean)
 
                    vis_filename = f"{drawing_id}_part{i+1}_visualization.tiff"
                    vis_path = os.path.join(masks_output_dir, vis_filename)

  
                    overlay = resized_image.copy()   
                
                    mask_indices = mask_clean > 0 
 
                    overlay[mask_indices] = np.array([0, 255, 0], dtype=overlay.dtype)  
 
                    cv2.circle(overlay, (int(scaled_x), int(scaled_y)), 10, (0, 0, 255), -1)  
 
                    alpha = 0.4
                    vis = cv2.addWeighted(resized_image, 1 - alpha, overlay, alpha, 0) 
 
                    cv2.imwrite(vis_path, vis)

                except Exception as e:
                    print(f"Error generating mask for {drawing_id}, point {i}: {e}")
                    import traceback
                    traceback.print_exc()
        except Exception as e:
            print(f"Error processing drawing {drawing_id}: {e}")
            traceback.print_exc()

In [ ]:
def main():
    check_cuda()
    print(f"Using device: {DEVICE}")

    optimize_cuda_memory()

    df = pd.read_csv(CSV_PATH)
    print(f"Loaded dataset with {len(df)} entries")
    
    train_csv = os.path.join(OUTPUT_DIR, "train_split.csv")
    val_csv = os.path.join(OUTPUT_DIR, "val_split.csv")
    
    if os.path.exists(train_csv) and os.path.exists(val_csv):
        print("Loading existing train/val split...")
        train_df = pd.read_csv(train_csv)
        val_df = pd.read_csv(val_csv)
    else:
        print("Creating new train/val split...")
        train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        train_df.to_csv(train_csv, index=False)
        val_df.to_csv(val_csv, index=False)
        
    print(f"Split dataset: {len(train_df)} training, {len(val_df)} validation")
 
    try:
        train_transform = A.Compose([
            A.Resize(IMAGE_SIZE[0], IMAGE_SIZE[1]),
            A.RandomRotate90(p=0.3),
            A.HorizontalFlip(p=0.3),
            A.VerticalFlip(p=0.3),
            A.GaussNoise(p=0.2),
            A.RandomBrightnessContrast(p=0.2),
        ], keypoint_params=A.KeypointParams(format='xy'), is_check_shapes=False)
        
        val_transform = A.Compose([
            A.Resize(IMAGE_SIZE[0], IMAGE_SIZE[1]),
        ], keypoint_params=A.KeypointParams(format='xy'), is_check_shapes=False)
    except ImportError:
        print("Albumentations not found. Using no transforms.")
        train_transform = None
        val_transform = None
        
    print("Creating datasets...")
 
    train_dataset = DrawingMaskDataset(
        train_csv, 
        DRAWINGS_DIR,
        MASKS_DIR,
        transform=train_transform
    )
    
    val_dataset = DrawingMaskDataset(
        val_csv,  
        DRAWINGS_DIR,
        MASKS_DIR,
        transform=val_transform
    )
     
    gradient_accumulation_steps = 4  
    use_gradient_checkpointing = True  
 
    fine_tuner = SAMFineTuner(
        SAM_CHECKPOINT, 
        MODEL_TYPE,
        gradient_accumulation_steps=gradient_accumulation_steps,
        use_gradient_checkpointing=use_gradient_checkpointing
    ) 

    embeddings_dir = os.path.join(OUTPUT_DIR, "embeddings")
    if not os.path.exists(embeddings_dir):
        os.makedirs(embeddings_dir, exist_ok=True)
        
 
    need_compute_embeddings = True
    if os.path.exists(embeddings_dir):
        embedding_files = [f for f in os.listdir(embeddings_dir) if f.endswith("_embedding.pt")]
        if len(embedding_files) >= len(train_dataset) + len(val_dataset) - 100: 
            need_compute_embeddings = False
            print(f"Found {len(embedding_files)} precomputed embeddings. Skipping computation.")
             
    if need_compute_embeddings:
        print("Computing embeddings...")
  
        compute_batch_size = BATCH_SIZE * 2
        
        precompute_train_loader = DataLoader(
            train_dataset,
            batch_size=compute_batch_size,
            shuffle=False,
            num_workers=0,
            pin_memory=True 
        )
        
        precompute_val_loader = DataLoader(
            val_dataset,
            batch_size=compute_batch_size,
            shuffle=False,
            num_workers=0,
            pin_memory=True
        )
 
        precompute_embeddings(precompute_train_loader, fine_tuner.sam, fine_tuner.device, embeddings_dir)
        precompute_embeddings(precompute_val_loader, fine_tuner.sam, fine_tuner.device, embeddings_dir)
  
    train_embeddings_dataset = EmbeddingsDataset(train_dataset, embeddings_dir)
    val_embeddings_dataset = EmbeddingsDataset(val_dataset, embeddings_dir)
    
    train_loader = DataLoader(
        train_embeddings_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=0,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_embeddings_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=0,
        pin_memory=True
    )

    print(f"Created dataloaders with {len(train_dataset)} training and {len(val_dataset)} validation samples")
    print(f"Effective batch size with gradient accumulation: {BATCH_SIZE * gradient_accumulation_steps}")
  
    fine_tuner.load_best_model()
    fine_tuner.train(train_loader, val_loader, NUM_EPOCHS)
 

if __name__ == "__main__":
    from contextlib import nullcontext
    main()

CUDA available: True
CUDA device count: 1
CUDA device name: NVIDIA GeForce RTX 3050 Ti Laptop GPU


NameError: name 'DEVICE' is not defined

In [11]:
gradient_accumulation_steps = 4 
use_gradient_checkpointing = True  

optimize_cuda_memory()

df = pd.read_csv(CSV_PATH)
fine_tuner = SAMFineTuner(
    SAM_CHECKPOINT, 
    MODEL_TYPE,
    gradient_accumulation_steps=gradient_accumulation_steps,
    use_gradient_checkpointing=use_gradient_checkpointing
    )
embeddings_dir = os.path.join(OUTPUT_DIR, "embeddings")

fine_tuner.load_best_model()
generate_all_masks(df, fine_tuner, embeddings_dir)

Applied aggressive memory optimizations
Initializing SAM fine tuner with memory optimizations...


C:\Users\user\AppData\Local\Temp\ipykernel_18152\1844730093.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Enabling gradient checkpointing...
Using gradient accumulation with 4 steps
SAM fine tuner initialized with memory optimizations
Clearing CUDA cache before loading model...
Attempting to load model state dict from C:/Users/user/Desktop/data/sam\best_sam_model.pth onto CPU...
State dict loaded onto CPU.
Loading state dict into model on cuda...
Loaded best model from C:/Users/user/Desktop/data/sam\best_sam_model.pth
Generating masks for all drawings...


Processing drawings:   0%|          | 5/1466 [00:00<00:32, 45.45it/s]

Mask for 01970 part 1 already exists. Skipping.
Mask for 032120 part 1 already exists. Skipping.
Mask for 032120 part 2 already exists. Skipping.
Mask for 03462 part 1 already exists. Skipping.
Mask for 03462 part 2 already exists. Skipping.
Mask for 0511.2075 part 1 already exists. Skipping.
Mask for 0511.2212 part 1 already exists. Skipping.
Mask for 058241 part 1 already exists. Skipping.
Mask for 058241 part 2 already exists. Skipping.
Mask for 058242 part 1 already exists. Skipping.
Mask for 058242 part 2 already exists. Skipping.
Mask for 064642 part 1 already exists. Skipping.
Mask for 064642 part 2 already exists. Skipping.


...

Processing drawings: 100%|█████████▉| 1461/1466 [00:27<00:00, 56.35it/s]

Mask for G645008-1 part 1 already exists. Skipping.
Mask for G645008-1 part 2 already exists. Skipping.
Mask for JG0BV092-6 part 1 already exists. Skipping.
Mask for JG0BV123-5 part 1 already exists. Skipping.
Mask for JG0BV148-6 part 1 already exists. Skipping.
Mask for JG0BV148-6 part 2 already exists. Skipping.
Mask for JG0BV152-5 part 1 already exists. Skipping.
Mask for JG0BV152-5 part 2 already exists. Skipping.
Mask for JG0RA084-2 part 1 already exists. Skipping.
Mask for JG0RA084-2 part 2 already exists. Skipping.
Mask for JG0RV276-9 part 1 already exists. Skipping.
Mask for JG0RV276-9 part 2 already exists. Skipping.
Selected cpu as target device for prediction


Processing drawings: 100%|█████████▉| 1465/1466 [00:55<00:01,  1.58s/it]

Mask for OOLE0210630 part 2 already exists. Skipping.
Selected cpu as target device for prediction


Processing drawings: 100%|██████████| 1466/1466 [01:30<00:00, 16.19it/s]
